# Fine Tuning
OpenAI's text generation models have been pre-trained on a vast amount of text. To use the models effectively, we include instructions and sometimes several examples in a prompt. Using demonstrations to show how to perform a task is often called "few-shot learning."

Fine-tuning improves on few-shot learning by training on many more examples than can fit in the prompt, letting you achieve better results on a wide number of tasks. Once a model has been fine-tuned, you won't need to provide as many examples in the prompt. This saves costs and enables lower-latency requests.

Fine-tuning lets you get more out of the models available through the API by providing:

* Higher quality results than prompting
* Ability to train on more examples than can fit in a prompt
* Token savings due to shorter prompts
* Lower latency requests

At a high level, fine-tuning involves the following steps:

* Prepare and upload training data* 
* Train a new fine-tuned model
* Evaluate results and go back to step 1 if needed
* Use your fine-tuned model

## When to use fine-tuning
Fine-tuning OpenAI text generation models can make them better for specific applications, but it requires a careful investment of time and effort. We recommend first attempting to get good results with prompt engineering, prompt chaining (breaking complex tasks into multiple prompts), and function calling, with the key reasons being:

* There are many tasks at which our models may not initially appear to perform well, but results can be improved with the right prompts - thus fine-tuning may not be necessary
* Iterating over prompts and other tactics has a much faster feedback loop than iterating with fine-tuning, which requires creating datasets and running training jobs
* In cases where fine-tuning is still necessary, initial prompt engineering work is not wasted - we typically see best results when using a good prompt in the fine-tuning data (or combining prompt chaining / tool use with fine-tuning)

In [1]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
_ : bool = load_dotenv(find_dotenv()) # 

client: OpenAI = OpenAI()

## Preparing your dataset
Once you have determined that fine-tuning is the right solution (i.e. you’ve optimized your prompt as far as it can take you and identified problems that the model still has), you’ll need to prepare data for training the model. You should create a diverse set of demonstration conversations that are similar to the conversations you will ask the model to respond to at inference time in production.

#### Upload file for fine tuning

In [2]:
file_name:str = "message.jsonl"
file_data = open(file_name, "rb")

file = client.files.create(
    file=file_data,
    purpose="fine-tune",
)

#### Create a fine-tune model

In [3]:
fine_tune_model = client.fine_tuning.jobs.create(
  training_file=file.id, 
  model="gpt-3.5-turbo"
)
print(fine_tune_model)

BadRequestError: Error code: 400 - {'error': {'message': 'Fine-tuning jobs cannot be created on an Explore plan. You can upgrade to a paid plan on your billing page: https://platform.openai.com/account/billing/overview', 'type': 'invalid_request_error', 'param': None, 'code': 'exceeded_quota'}}

#### It took more than an hour in my case

In [ ]:
import time
# Retrieve the state of a fine-tune
while True:
    data = client.fine_tuning.jobs.retrieve(fine_tune_model.id)

    if data.status == "succeeded":
        break
    elif data.status == "failed":
        print("Fine-tune failed")
        break
    else:
        print("Fine-tune still running")
        time.sleep(5)

In [ ]:
from openai.types.chat.chat_completion import ChatCompletion

def chat_completion(prompt : str )-> str:
 response : ChatCompletion = client.chat.completions.create(
        model= "ft:gpt-3.5-turbo-0613:personal::8TEbB8a7",
        messages=[
            {"role": "system", "content": "You are a sarcastic assistant."},
            {
                "role": "user",
                "content": prompt,
            }
        ],
    )
 return response.choices[0].message.content

In [ ]:
chat_completion("Who wrote 'Romeo and Juliet'?")

In [ ]:
chat_completion("who was John F. Kennedy?")

In [ ]:
chat_completion("are you a fine-tuned model?")

In [ ]:
chat_completion("what is your purpose?")

In [ ]:
chat_completion("Write some sentences as Imran Khan.")

In [ ]:
chat_completion("Write some sentences as Joe Biden.")